In [1]:
import pandas as pd

import sys
sys.path.append('../') # Root of the repo

from helpers import sparqlToDataframe, sparqlToTermDict, wordsToSingleToken

In [2]:
subpart_qry = """
# This query gets right lung and left lung children and grand children
# And maps back to RIGHT_LUNG or LEFT_LUNG
PREFIX radlex: <http://www.radlex.org/RID/#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?rid ?seed_name ?name ?synonym 
WHERE {
  {
    SELECT ?rid ?seed_name WHERE {
      VALUES ?seed { <http://www.radlex.org/RID/#RID1302> <http://www.radlex.org/RID/#RID1326> }
      ?seed radlex:Preferred_name ?seed_name .
      ?seed radlex:Has_Part{0,2} ?rid
    }
  } .
  ?rid radlex:Preferred_name ?name .
  OPTIONAL{?rid radlex:Synonym ?synonym  }
} ORDER BY ?seed_name
"""

In [3]:
subpartSparql_df = sparqlToTermDict(subpart_qry, keyColumn='name', nameCols=[ 'name', 'synonym' ])

In [4]:
subpartKnowledge_df = pd.DataFrame([
    { 'meanings': 'lt lung', 'keys' : 'LEFT_LUNG'  },
    { 'meanings': 'rt lung', 'keys' : 'RIGHT_LUNG' },
])

In [5]:
subpart_df = pd.concat([subpartSparql_df, subpartKnowledge_df], ignore_index=True, sort=True)

In [9]:
subpart_df.to_csv('../Data/clever_ext_lung_synonyms.txt',sep='|',index_label='id', 
                    columns=['meanings', 'keys'])